In [25]:
import pandas as pd
import torch as t
import numpy as np

# LM-37 bacteria

In [6]:
df = pd.read_csv(
    "bacteria-37.csv",
    header=1,
    usecols= [
        "Position X",
        "Position Y",
        "Position Z",
        "Time",
        "TrackID",
        "ID"
    ]
)

In [11]:
display(df.head())
df.info()

,Position X,Position Y,Position Z,Time,TrackID,ID
0,248.6320,10.7085,0.1815,1,1000000008,8
1,346.6570,10.7085,0.1815,1,1000000014,14
2,52.1679,11.1991,0.1815,1,1000000015,15
3,32.9705,11.6746,0.1815,1,1000000018,18
4,272.7170,11.8309,0.1815,1,1000000020,20


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17822 entries, 0 to 17821
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Position X  17822 non-null  float64
 1   Position Y  17822 non-null  float64
 2   Position Z  17822 non-null  float64
 3   Time        17822 non-null  int64  
 4   TrackID     17822 non-null  int64  
 5   ID          17822 non-null  int64  
dtypes: float64(3), int64(3)
memory usage: 835.5 KB


In [13]:
pd.unique(df["TrackID"]).shape

(299,)

In [28]:
(df[df["TrackID"] == 1000000015]).sort_values(by="Time")

,Position X,Position Y,Position Z,Time,TrackID,ID
2,52.1679,11.1991,0.1815,1,1000000015,15
298,52.1539,11.2009,0.1815,2,1000000015,1557
595,52.1554,11.2338,0.1815,3,1000000015,3272
893,52.2164,11.2702,0.1815,4,1000000015,4848
1191,52.1607,11.2256,0.1815,5,1000000015,6400
1489,52.2040,11.2801,0.1815,6,1000000015,8034
1787,52.1384,11.2526,0.1815,7,1000000015,9610
2085,52.2176,11.2861,0.1815,8,1000000015,11335
2383,52.1484,11.2188,0.1815,9,1000000015,13067
2680,52.1507,11.1809,0.1815,10,1000000015,14778


In [26]:
np.unique(df["Time"].to_numpy(), return_counts=True)

(array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
        18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
        35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51,
        52, 53, 54, 55, 56, 57, 58, 59, 60], dtype=int64),
 array([296, 297, 298, 298, 298, 298, 298, 298, 297, 297, 297, 297, 297,
        297, 297, 297, 297, 297, 297, 297, 297, 297, 297, 297, 297, 297,
        297, 297, 297, 297, 297, 297, 297, 297, 297, 297, 296, 296, 296,
        297, 297, 297, 297, 297, 297, 297, 297, 297, 297, 297, 297, 297,
        297, 297, 297, 297, 297, 297, 297, 297], dtype=int64))

In [32]:
tracks = pd.unique(df["TrackID"])

x_fluctuations = []
y_fluctuations = []
for track_id in tracks:
    pos_x = df[df["TrackID"] == track_id]["Position X"]
    pos_y = df[df["TrackID"] == track_id]["Position Y"]
    x_fluctuations.append(pos_x.max() - pos_x.min())
    y_fluctuations.append(pos_y.max() - pos_y.min())
print(f"min. x fluctuation: {np.min(x_fluctuations)}, max. x fluctuation: {np.max(x_fluctuations)}, avg. x fluctuation: {np.mean(x_fluctuations)}")
print(f"min. y fluctuation: {np.min(y_fluctuations)}, max. y fluctuation: {np.max(y_fluctuations)}, avg. y fluctuation: {np.mean(y_fluctuations)}")    

min. x fluctuation: 0.0, max. x fluctuation: 0.8780000000000143, avg. x fluctuation: 0.06948056856187337
min. y fluctuation: 0.0, max. y fluctuation: 0.30449999999999733, avg. y fluctuation: 0.06715183946488228


# LM-RT bacteria

In [36]:
df_rt = pd.read_csv(
    "bacteria-RT.csv",
    header=1,
    usecols= [
        "Position X",
        "Position Y",
        "Position Z",
        "Time",
        "TrackID",
        "ID"
    ]
)

In [37]:
display(df_rt.head())
df_rt.info()

,Position X,Position Y,Position Z,Time,TrackID,ID
0,295.338,258.637,0.1815,1,1000000007,7
1,422.484,258.637,0.1815,1,1000000014,14
2,468.483,258.637,0.1815,1,1000000016,16
3,362.448,259.978,0.1815,1,1000000025,25
4,470.481,261.132,0.1815,1,1000000029,29


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29493 entries, 0 to 29492
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Position X  29493 non-null  float64
 1   Position Y  29493 non-null  float64
 2   Position Z  29493 non-null  float64
 3   Time        29493 non-null  int64  
 4   TrackID     29493 non-null  int64  
 5   ID          29493 non-null  int64  
dtypes: float64(3), int64(3)
memory usage: 1.4 MB


In [38]:
np.unique(df_rt["Time"].to_numpy(), return_counts=True)

(array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
        18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
        35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51,
        52, 53, 54, 55, 56, 57, 58, 59, 60], dtype=int64),
 array([472, 479, 488, 491, 488, 490, 493, 494, 494, 498, 496, 490, 488,
        486, 488, 486, 491, 492, 495, 496, 498, 495, 489, 493, 491, 491,
        485, 486, 488, 495, 491, 501, 495, 497, 496, 497, 497, 494, 493,
        491, 496, 494, 498, 500, 498, 502, 499, 498, 492, 490, 488, 493,
        489, 494, 491, 495, 496, 483, 476, 463], dtype=int64))

In [41]:
tracks = pd.unique(df_rt["TrackID"])

x_fluctuations = []
y_fluctuations = []
for track_id in tracks:
    pos_x = df_rt[df_rt["TrackID"] == track_id]["Position X"]
    pos_y = df_rt[df_rt["TrackID"] == track_id]["Position Y"]
    x_fluctuations.append(pos_x.max() - pos_x.min())
    y_fluctuations.append(pos_y.max() - pos_y.min())
print(f"min. x fluctuation: {np.min(x_fluctuations)}, max. x fluctuation: {np.max(x_fluctuations)}, avg. x fluctuation: {np.mean(x_fluctuations)}")
print(f"min. y fluctuation: {np.min(y_fluctuations)}, max. y fluctuation: {np.max(y_fluctuations)}, avg. y fluctuation: {np.mean(y_fluctuations)}")    

min. x fluctuation: 0.0, max. x fluctuation: 133.238, avg. x fluctuation: 15.301285886610373
min. y fluctuation: 0.0, max. y fluctuation: 205.789, avg. y fluctuation: 15.736419782870927


# Bacterial CPM

- either 1 or 2 pixels (2-4 $\mu m$) -> 1 cell is 2 Microns, the unit in the df for positions is Microns
- 